In [1]:
import os 
import pandas as pd

## **GF Project 6**

**Project Goal and Objectives**: 1) Give previous day stats on cards & 2) Report comaparisons to moving averages

### **Bot**:
 - 1) Trigger the file request using a scheduler: 00:01 EST and every 30 mins comapring to previous same date 2 week MA. 
 - 2) Create the function which: 
        1 -  calls the Playwright Function to retrieve the files
        2 -  calls the notebook
        3 -  obtains the message 
 - 3) Output the print message from the notebook to the chat
 - 4) Keep in the directory the files shown in ORIGINAL_FILES list

### **NEF_Project_7_local.ipynb**:
- 1) Data preprocessing: clean data, handle missing values, standardize formats
- 2) Will start by importing the files 
- 3) Retreieve all the stats for the previous day
- 4) Filter by chareg type and then by cards
- 5) Insert values to the database 
- 6) Comapre result with MAs from same time of previous last week, if standard deviaiton >20% --> ALERT 
- 7) Convert the stats to a message, by CC type, them by type of charge. Share both totals and margins (%) 


### **DATABASE(S)**:
- 1) We are going to create a database for each card type;
- 2) Columns: Successful Captured Charges and Successful Capture Rate


### Import the files needed 

In [2]:
# STeps case these are empty
import pandas as pd
from pandas.errors import EmptyDataError

# Define your columns
columns = [
    'Card Type', 'Customers', 'Attempted Captured Charges',
    'Declined Captured Charges', 'Decline Capture Rate',
    'Declined Net Product Revenue', 'Successful Captured Charges',
    'Successful Capture Rate', 'Success Gross Product Revenue',
    'Rewards Applied', 'Discounts Applied', 'Discount Rate',
    'Successful Net Product Revenue', 'Shipping Revenue', 'Tax Collected',
    'Gift Cards Applied', 'Revenue', 'Processed', 'Adjustments',
    'Adjustment Rate', 'Successful Refunds', 'Declined Refunds',
    'Refund Rate', 'Refunded Revenue', 'Refunded Revenue Rate',
    'Full Refunds', 'Full Refund Rate', 'Partial Refunds',
    'Partial Refund Rate', 'Successful Voids', 'Declined Voids',
    'Void Rate', 'Voided Revenue', 'Voided Revenue Rate', 'Chargebacks',
    'Chargeback Rate', 'Chargeback Revenue', 'Chargeback Revenue Rate',
    'Alerts', 'Alert Rate', 'Alert Revenue', 'Alert Revenue Rate',
    'Net Revenue', 'Cost of Revenue', 'Cost of Goods Sold', 'Ad Spend',
    'Chargeback Fees', 'Alert Fees', 'Processing Fees', 'Profit',
    'Profit Margin'
]

def safe_read_csv(path):
    try:
        df = pd.read_csv(path)
        # If the file is empty but has headers, this will still work
        if df.empty:
            raise EmptyDataError
        return df
    except (EmptyDataError, pd.errors.ParserError):
        # Create a blank DataFrame with the correct columns and 2 rows for card types
        df = pd.DataFrame({
            'Card Type': ['Mastercard', 'Visa'],
        })
        for col in columns:
            if col != 'Card Type':
                df[col] = 0
        return df[columns]  # Ensure column order

# Usage
s1a = safe_read_csv("S1R1A1.csv")
s1aa = safe_read_csv("S1R2+A1.csv")
s2a = safe_read_csv("S2R1A1.csv")
s2aa = safe_read_csv("S2R2+A1.csv")

list_of_dfs = [s1a, s1aa, s2a, s2aa]

#### Print each of these 

In [3]:
s1a

,Card Type,Customers,Attempted Captured Charges,Declined Captured Charges,Decline Capture Rate,Declined Net Product Revenue,Successful Captured Charges,Successful Capture Rate,Success Gross Product Revenue,Rewards Applied,...,Alert Revenue Rate,Net Revenue,Cost of Revenue,Cost of Goods Sold,Ad Spend,Chargeback Fees,Alert Fees,Processing Fees,Profit,Profit Margin
0,Mastercard,57,39,30,76.92%,"$2,966.70",9,23.08%,$890.01,$0.00,...,200.00%,$-890.01,$52.26,$0.00,$0.00,$0.00,$0.00,$52.26,$-942.27,-105.87%
1,Visa,89,88,79,89.77%,"$7,812.31",9,10.23%,$890.01,$0.00,...,0.00%,$855.40,$62.06,$0.00,$0.00,$0.00,$0.00,$62.06,$793.34,89.14%


In [4]:
s1aa

,Card Type,Customers,Attempted Captured Charges,Declined Captured Charges,Decline Capture Rate,Declined Net Product Revenue,Successful Captured Charges,Successful Capture Rate,Success Gross Product Revenue,Rewards Applied,...,Alert Revenue Rate,Net Revenue,Cost of Revenue,Cost of Goods Sold,Ad Spend,Chargeback Fees,Alert Fees,Processing Fees,Profit,Profit Margin
0,Mastercard,14,13,11,84.62%,"$1,087.79",2,15.38%,$197.78,$0.00,...,50.00%,$98.89,$12.48,$0.00,$0.00,$0.00,$0.00,$12.48,$86.41,43.69%
1,Visa,9,9,9,100.00%,$890.01,0,0.00%,$0.00,$0.00,...,0.00%,$0.00,$1.80,$0.00,$0.00,$0.00,$0.00,$1.80,$-1.80,0.00%


In [5]:
s2a

,Card Type,Customers,Attempted Captured Charges,Declined Captured Charges,Decline Capture Rate,Declined Net Product Revenue,Successful Captured Charges,Successful Capture Rate,Success Gross Product Revenue,Rewards Applied,...,Alert Revenue Rate,Net Revenue,Cost of Revenue,Cost of Goods Sold,Ad Spend,Chargeback Fees,Alert Fees,Processing Fees,Profit,Profit Margin
0,Mastercard,9,6,4,66.67%,$191.36,2,33.33%,$95.68,$0.00,...,150.00%,$-47.84,$5.98,$0.00,$0.00,$0.00,$0.00,$5.98,$-53.82,-56.25%
1,Visa,3,3,2,66.67%,$95.68,1,33.33%,$47.84,$0.00,...,0.00%,$47.84,$2.99,$0.00,$0.00,$0.00,$0.00,$2.99,$44.85,93.75%


In [6]:
s2aa

,Card Type,Customers,Attempted Captured Charges,Declined Captured Charges,Decline Capture Rate,Declined Net Product Revenue,Successful Captured Charges,Successful Capture Rate,Success Gross Product Revenue,Rewards Applied,...,Alert Revenue Rate,Net Revenue,Cost of Revenue,Cost of Goods Sold,Ad Spend,Chargeback Fees,Alert Fees,Processing Fees,Profit,Profit Margin
0,Mastercard,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Visa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Step 1) Data Preprocessing 


In [7]:
s1a = s1a[['Card Type', 'Customers', 'Successful Capture Rate']]
s1aa = s1aa[['Card Type', 'Customers', 'Successful Capture Rate']]
s2a = s2a[['Card Type', 'Customers', 'Successful Capture Rate']]
s2aa = s2aa[['Card Type', 'Customers', 'Successful Capture Rate']]


In [8]:
# Convert 'Attempted Captured Charges' to integer for each DataFrame
s1a['Successful Capture Rate'] = s1a['Successful Capture Rate'].astype(str).str.replace('%', '').astype(float)
s1aa['Successful Capture Rate'] = s1aa['Successful Capture Rate'].astype(str).str.replace('%', '').astype(float)
s2a['Successful Capture Rate'] = s2a['Successful Capture Rate'].astype(str).str.replace('%', '').astype(float)
s2aa['Successful Capture Rate'] = s2aa['Successful Capture Rate'].astype(str).str.replace('%', '').astype(float)

# Convert 'Customers' to integer and 'Card Type' to string for each DataFrame
s1a['Customers'] = s1a['Customers'].astype(int)
s1a['Card Type'] = s1a['Card Type'].astype(str)

s1aa['Customers'] = s1aa['Customers'].astype(int)
s1aa['Card Type'] = s1aa['Card Type'].astype(str)

s2a['Customers'] = s2a['Customers'].astype(int)
s2a['Card Type'] = s2a['Card Type'].astype(str)

s2aa['Customers'] = s2aa['Customers'].astype(int)
s2aa['Card Type'] = s2aa['Card Type'].astype(str)

C:\Users\mmsou\AppData\Local\Temp\ipykernel_23680\2219152271.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1a['Successful Capture Rate'] = s1a['Successful Capture Rate'].astype(str).str.replace('%', '').astype(float)
C:\Users\mmsou\AppData\Local\Temp\ipykernel_23680\2219152271.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1aa['Successful Capture Rate'] = s1aa['Successful Capture Rate'].astype(str).str.replace('%', '').astype(float)
C:\Users\mmsou\AppData\Local\Temp\ipykernel_23680\2219152271.p

#### Step 3 - Retrieve stats from previous day  

In [9]:
s1a_current_dict = {}
s1aa_current_dict = {}
s2a_current_dict = {}
s2aa_current_dict = {}



In [10]:
# For s1a
for row in s1a.itertuples(index=False):
    s1a_current_dict[row._0] = {"Customers": row.Customers, "ACC": row._2}

# For s1aa
for row in s1aa.itertuples(index=False):
    s1aa_current_dict[row._0] = {"Customers": row.Customers, "ACC": row._2}

# For s2a
for row in s2a.itertuples(index=False):
    s2a_current_dict[row._0] = {"Customers": row.Customers, "ACC": row._2}

# For s2aa
for row in s2aa.itertuples(index=False):
    s2aa_current_dict[row._0] = {"Customers": row.Customers, "ACC": row._2}

### Step 4 - Filter charge type and then by cards 

In [11]:
from datetime import datetime
import pytz

# Define the EST timezone
est = pytz.timezone('US/Eastern')

# Get the current time in EST
now_est = datetime.now(est)

# Format as hour:minute am/pm (e.g., 03:45 PM)
formatted_time = now_est.strftime("%I:%M %p")

time_in_ny = formatted_time
time_in_ny

'01:43 PM'

In [12]:
message = f"💳 A1RETENTION (NY: {time_in_ny})\n\n"

# Dicts with teh current values
current_dicts = [s1a_current_dict, s1aa_current_dict, s2a_current_dict, s2aa_current_dict]

# The names of the categories 
file_names = ["S1R1A1", "S1R2+A1", "S2R1A1", "S2R2+A1"]

for i in range(len(current_dicts)):
    message += f"• {file_names[i]}: \n"
    
    # Check if Visa exists in the current dictionary
    if 'Visa' in current_dicts[i]:
        message += f"→ Visa: {current_dicts[i]['Visa']['ACC']}% ({current_dicts[i]['Visa']['Customers']})\n"
    
    # Check if Mastercard exists in the current dictionary
    if 'Mastercard' in current_dicts[i]:
        message += f"→ MasterCard: {current_dicts[i]['Mastercard']['ACC']}% ({current_dicts[i]['Mastercard']['Customers']})\n"
    
    message += "\n"

print(message)

💳 A1RETENTION (NY: 01:43 PM)

• S1R1A1: 
→ Visa: 10.23% (89)
→ MasterCard: 23.08% (57)

• S1R2+A1: 
→ Visa: 0.0% (9)
→ MasterCard: 15.38% (14)

• S2R1A1: 
→ Visa: 33.33% (3)
→ MasterCard: 33.33% (9)

• S2R2+A1: 
→ Visa: 0.0% (0)
→ MasterCard: 0.0% (0)




#### Step 5 - Adding values to the Database


In [13]:
import pandas as pd
import pandas as pd

a1db = pd.read_excel("a1_master_database_gf.xlsx")


In [14]:
from datetime import datetime, timedelta

def get_current_rounded_time(x):
    """
    Get current time rounded to nearest half hour.
    Returns datetime object rounded to nearest half hour.
    """
    now = x
    
    # Calculate minutes to add/subtract to round to nearest half hour
    minutes = now.minute
    if minutes < 15:
        # Round down to previous half hour
        rounded_time = now.replace(minute=0, second=0, microsecond=0)
    elif minutes < 45:
        # Round to current half hour
        rounded_time = now.replace(minute=30, second=0, microsecond=0)
    else:
        # Round up to next half hour
        rounded_time = now.replace(minute=0, second=0, microsecond=0) + timedelta(hours=1)
    
    return rounded_time
timey = datetime.now()
datetimee = get_current_rounded_time(timey)
print(datetimee)

2025-06-25 18:30:00


In [15]:
from datetime import datetime
from zoneinfo import ZoneInfo

now_est = datetime.now(ZoneInfo("America/New_York"))
# If you have a function that returns a datetime, e.g. get_current_rounded_time(now_est)
est_time = get_current_rounded_time(now_est)
print(est_time.strftime("%Y-%m-%d %H:%M:%S"))

2025-06-25 13:30:00


In [16]:
# Before the loop, add this debug code
print("First dictionary structure:", current_dicts[0].keys() if current_dicts else "No dictionaries")

First dictionary structure: dict_keys(['Mastercard', 'Visa'])


In [17]:
current_dicts

[{'Mastercard': {'Customers': 57, 'ACC': 23.08},
  'Visa': {'Customers': 89, 'ACC': 10.23}},
 {'Mastercard': {'Customers': 14, 'ACC': 15.38},
  'Visa': {'Customers': 9, 'ACC': 0.0}},
 {'Mastercard': {'Customers': 9, 'ACC': 33.33},
  'Visa': {'Customers': 3, 'ACC': 33.33}},
 {'Mastercard': {'Customers': 0, 'ACC': 0.0},
  'Visa': {'Customers': 0, 'ACC': 0.0}}]

In [18]:
for i in range(len(current_dicts)):
    # Always add Mastercard row if present
    if "Mastercard" in current_dicts[i]:
        new_row1 = {
            "Class": file_names[i],
            "Card Type": "MasterCard",
            "Rate": current_dicts[i]["Mastercard"]["ACC"],
            "Customers": current_dicts[i]["Mastercard"]["Customers"],
            "Date-Time": est_time,
        }
        a1db.loc[len(a1db)] = new_row1

    # Only add Visa row if present
    if "Visa" in current_dicts[i]:
        new_row2 = {
            "Class": file_names[i],
            "Card Type": "Visa",
            "Rate": current_dicts[i]["Visa"]["ACC"],
            "Customers": current_dicts[i]["Visa"]["Customers"],
            "Date-Time": est_time,
        }
        a1db.loc[len(a1db)] = new_row2
    else:
        print(f"Warning: No 'Visa' key in current_dicts[{i}]")

In [19]:
a1db['Date-Time'] = pd.to_datetime(a1db['Date-Time']).apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))


C:\Users\mmsou\AppData\Local\Temp\ipykernel_23680\2340810370.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  a1db['Date-Time'] = pd.to_datetime(a1db['Date-Time']).apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))


In [20]:
import os

# Delete the existing file if it exists
file_path = "a1_master_database_gf.xlsx"
if os.path.exists(file_path):
    os.remove(file_path)

# Save the updated DataFrame as the new file
a1db.to_excel(file_path, index=False)